In [ ]:
from flask import Flask, render_template_string, request
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import threading
import webbrowser

# ======== Load & Prepare Data ========
heart_data = pd.read_csv("heart_disease_data.csv")

X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, Y_train)

train_pred = model.predict(X_train_scaled)
test_pred = model.predict(X_test_scaled)

training_data_accuracy = accuracy_score(Y_train, train_pred)
testing_data_accuracy = accuracy_score(Y_test, test_pred)

print("Train Accuracy:", training_data_accuracy)
print("Test Accuracy :", testing_data_accuracy)
print("\nClassification Report:\n", classification_report(Y_test, test_pred))

# ======== Flask App ========
app = Flask(__name__)

form_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Heart Disease Prediction</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(to right, #a1c4fd, #c2e9fb);
            margin: 0;
            padding: 0;
            color: #333;
        }
        .container {
            max-width: 600px;
            margin: 60px auto;
            background: #ffffffcc;
            padding: 40px;
            border-radius: 12px;
            box-shadow: 0 6px 25px rgba(0, 0, 0, 0.15);
        }
        h2 {
            text-align: center;
            color: #2c3e50;
            margin-bottom: 30px;
        }
        label {
            display: block;
            margin-top: 15px;
            color: #34495e;
            font-weight: 500;
        }
        input[type="text"] {
            width: 100%;
            padding: 10px;
            margin-top: 5px;
            border: 1px solid #ccc;
            border-radius: 6px;
            background-color: #f9f9f9;
            font-size: 14px;
            color: #2c3e50;
        }
        input[type="submit"] {
            margin-top: 25px;
            width: 100%;
            padding: 12px;
            background-color: #2980b9;
            color: #fff;
            border: none;
            border-radius: 8px;
            font-size: 16px;
            font-weight: bold;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background-color: #1f6391;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Heart Disease Prediction Form</h2>
        <form action="/predict" method="post">
            {% for label, name in fields %}
                <label>{{ label }}:</label>
                <input type="text" name="{{ name }}">
            {% endfor %}
            <input type="submit" value="Predict">
        </form>
    </div>
</body>
</html>
'''

result_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Prediction Result</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(to right, #fbc2eb, #a6c1ee);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            color: #2c3e50;
        }
        .result-box {
            background: #ffffffcc;
            padding: 40px;
            border-radius: 14px;
            text-align: center;
            box-shadow: 0 6px 20px rgba(0, 0, 0, 0.15);
            max-width: 500px;
        }
        h2 {
            color: #e74c3c;
            font-size: 26px;
        }
        p {
            font-size: 18px;
            margin-top: 15px;
            color: #34495e;
        }
        a {
            display: inline-block;
            margin-top: 25px;
            padding: 10px 20px;
            background-color: #3498db;
            color: white;
            text-decoration: none;
            border-radius: 8px;
            font-weight: bold;
        }
        a:hover {
            background-color: #217dbb;
        }
    </style>
</head>
<body>
    <div class="result-box">
        <h2>{{ prediction }}</h2>
        <p>Model Training Accuracy: {{ accuracy }}%</p>
        <a href="/">← Back to Home</a>
    </div>
</body>
</html>
'''

fields = [
    ('Age', 'age'),
    ('Sex (1=Male, 0=Female)', 'sex'),
    ('Chest Pain Type (0-3)', 'cp'),
    ('Resting Blood Pressure', 'trestbps'),
    ('Cholesterol', 'chol'),
    ('Fasting Blood Sugar > 120 (1 = yes)', 'fbs'),
    ('Rest ECG (0,1,2)', 'restecg'),
    ('Max Heart Rate', 'thalach'),
    ('Exercise Induced Angina (1 = yes)', 'exang'),
    ('Oldpeak', 'oldpeak'),
    ('Slope (0-2)', 'slope'),
    ('Number of Major Vessels (0-3)', 'ca'),
    ('Thal (1=normal, 2=fixed, 3=reversible)', 'thal'),
]

@app.route('/')
def home():
    return render_template_string(form_html, fields=fields)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        input_data = [float(request.form[field[1]]) for field in fields]
        input_array = np.asarray(input_data).reshape(1, -1)
        input_scaled = scaler.transform(input_array)
        prediction = model.predict(input_scaled)

        if prediction[0] == 0:
            result = 'The Person does NOT have Heart Disease'
        else:
            result = 'The Person HAS Heart Disease'

        return render_template_string(result_html, prediction=result, accuracy=round(training_data_accuracy * 100, 2))
    
    except Exception as e:
        return f"<h3 style='color:red;'>Error: {e}. Please enter valid numerical values.</h3>"

# ======== Run Server ========
def run_flask():
    app.run(port=5000, debug=False)

# Uncomment to auto-launch in browser
# threading.Timer(1.25, lambda: webbrowser.open("http://127.0.0.1:5000")).start()

run_flask()
